# Lab 11: Bootstrapping and Confidence Intervals

Bootstrapping is a very popular method of inference that avoids making assumptions about the shape of sampling distributions. What it lacks in efficiency, it makes up for in rigor. Let's try it out!

Please complete this lab by providing answers in cells after the question. Use **Code** cells to write and run any code you need to answer the question and **Markdown** cells to write out answers in words. After you are finished with the assignment, remember to download it as an **.html file** and submit it in **ELMS**.

In [2]:
# These lines import the Numpy and Datascience modules.
import numpy as np
import datascience as ds

# These lines do some fancy plotting magic
import matplotlib
# Required to view plots in a notebook
%matplotlib inline
import matplotlib.pyplot as plt
# This is just to make the plots look a certain way
plt.style.use('fivethirtyeight')

## Motivating Question

Suppose we wanted to know the answer to the following question:

**What is the median salary of all employees of the City of San Francisco?**

Note that we aren't interested in whether the median salary is higher or lower than anything. We just wanted to know what the value is. This might be useful for getting an idea of what city employees generally make. Using a mean might not be the best measure, because we expect the salaries to be heavily right skewed, with a few people making much more than most.

## City of San Francisco Employee Salaries

The City of San Francisco makes the salary and compensation information for all of their employees available publicly through the [SF Open Data website](https://datasf.org/opendata/). We have pulled the 2015 data inside `san_francisco_2015.csv`. 

We will treat this full data set as the population. (This is for illustration. Remember, in practice, we almost never have access to the population.) In general, when creating confidence intervals, you will only have access to one sample such as the `our_sample` object created below. In this lab, we are using the full population data for demonstrative purposes to explore the properties of confidence intervals.

Let's start by bringing in the population data and taking a look at the median Total Salary. We remove the rows with salaries below 10,000 to make sure we aren't capturing people who may be recorded with very low values because they were employed only for a short period of time. 

In [3]:
population = ds.Table.read_table('san_francisco_2015.csv').where('Total Salary', ds.are.above(10000))
population.show(3)

Year Type,Year,Organization Group Code,Organization Group,Department Code,Department,Union Code,Union,Job Family Code,Job Family,Job Code,Job,Employee Identifier,Salaries,Overtime,Other Salaries,Total Salary,Retirement,Health/Dental,Other Benefits,Total Benefits,Total Compensation
Calendar,2015,2,"Public Works, Transportation & Commerce",WTR,PUC Water Department,21,"Prof & Tech Engineers - Miscellaneous, Local 21",2400,"Lab, Pharmacy & Med Techs",2481,Water Qualitytech I/II,21538,82146,0,0,82146,16942.2,12340.9,6337.73,35620.8,117767
Calendar,2015,2,"Public Works, Transportation & Commerce",DPW,General Services Agency - Public Works,12,"Carpet, Linoleum and Soft Tile Workers, Local 12",7300,Journeyman Trade,7393,Soft Floor Coverer,5459,32165.8,973.19,848.96,33987.9,0,4587.51,2634.42,7221.93,41209.8
Calendar,2015,4,Community Health,DPH,Public Health,790,"SEIU - Miscellaneous, Local 1021",1600,"Payroll, Billing & Accounting",1636,Health Care Billing Clerk 2,41541,71311,5757.98,0,77069,14697.6,12424.5,6370.06,33492.2,110561


We can look at the histogram of Total Salary to get an idea of what the population distribution looks like. We also find the median. This is the "unknown" population value that we will try to estimate using confidence intervals.

In [ ]:
pop_median = np.median(population.column('Total Salary'))
sf_bins = np.arange(0, 700000, 25000)
population.hist('Total Salary', bins=sf_bins)
print("Population Median = ", pop_median)

## Taking a Sample

If, as is usually the case, we did not have access to the full population, we might have taken a sample and tried to estimate the population median. Let's say we went out and took a simple random sample of 300 employees of the City of San Francisco and asked them what their salary was. We'll show what this process might have been like by using `.sample()`. Note that we are using `with_replacement = False` because we are simply showing what might have happened if we actually went out into the physical world and surveyed these people. 

In [ ]:
### Random sample of size 300
population_salary = population.select('Total Salary')
our_sample = population_salary.sample(300, with_replacement = False)
our_sample_median = np.median(our_sample.column('Total Salary'))
our_sample.hist('Total Salary', bins=sf_bins)
print("Sample Median = ", our_sample_median)

We will use `our_sample` as the sample that we use to estimate the median salary. Above, we calculated the **point estimate**—the single number that acts as our estimate—using the sample median. This is a reasonable estimate of the population median, but we might not be sure about how close we are the actual population median. In order to figure out our margin of error of this estimate, we can use bootstrap samples. A bootstrap sample is a random sample from the original data of the same size with replacement. If there were 100 people in the sample originally, I would randomly sample 100 observations from the dataset, and since it is with replacement, some rows will be sampled more than once and some not at all in a particular bootstrap sample (at least 1,000). Typically, you want to collect many bootstrap samples to generate an empirical sampling distribution. The idea is that the original sample resembles the population. If this is true, then you can use the original sample as the sampling frame, draw the 1,000+ bootstrap samples with replacement, and compute a statistic of interest in each resample (here, the median). This will leave you with 1,000+ medians that form the empirical sampling distribution of the median. (This approximates what we may have observed had we actually conducted 1,000+ studies of the same size and drawn data from the same population, computing the median in each.) We can use this empirical sampling distribution to construct confidence intervals for inference (e.g., about whether the population median is different from zero). You can create a bootstrap confidence interval a number of ways, but the most popular way is to sort the distribution of estimates and take, if it's a 95% confidence interval, the 2.5th and 97.5th percentile values and observing whether zero is between the values. If not, then you would reject the null hypothesis. 

First, let's take one bootstrap sample. Use `.sample()` to take a single bootstrap sample.

<font color = 'red'>**Question 1: Using `our_sample`, take one bootstrap sample of `Total Salary` assign it to `one_bootstrap_sample`.**</font>



In [ ]:
one_bootstrap_sample = ...
np.median(one_bootstrap_sample.column('Total Salary'))

Next, we can take the code above and define a function that takes a bootstrap sample and calculates the median of that bootstrap sample. This will let us use a loop to generate many bootstrap samples, and help us create our confidence interval. 

The next few questions walk through this process of generating a confidence interval. Remember, the steps in this simulation process are similar to a hypothesis test:
- Define a function that performs the simulation (the bootstrap sample) and calculates the statistic (median).
- Use a loop to do this simulation many times, storing the statistic of each iteration in an array (`bootstrap_medians`).
- Use the statistics collected from the simulation to make your conclusion.

<font color = 'red'>**Question 2: Next, create a function called `one_bootstrap_median` that takes a bootstrap sample, then calculates the median of `Total Salary`. This function should return one value, the median of the one bootstrap sample that we take.**</font>

*Hint: You should be using a lot of the same code as above.*

In [ ]:
def one_bootstrap_median(samp):
    ...
    return ...

one_bootstrap_median(our_sample)

<font color = 'red'>**Question 3: Use the `one_bootstrap_median` function and a `for` loop to generate 1000 bootstrap samples and assign the bootstrap medians to an array called `bootstrap_medians`.**</font>

*Hint: The loop should be very similar to what we've done in hypothesis tests.*

In [ ]:
bootstrap_medians = ds.make_array()
num_bootstraps = 1000

for ... in ...:
    ...

<font color = 'red'>**Question 4: Find the 95% confidence interval. Assign the left bound to `left` and the right bound to `right`.**</font>

*Hint: Remember, the `ds.percentile` function can help you find the middle 95%*



In [ ]:
left = ...
right = ...
print('We are 95% confident that the median salary of all SF City employees is between', left, 'and', right)

We can visualize the confidence interval on the histogram. The green point is the population median (the true value that we are trying to estimate) for reference.

In [ ]:
med_bins = np.arange(65000, 90001, 2500)
Table().with_column('Bootstrap Medians', bootstrap_medians).hist('Bootstrap Medians', bins=med_bins)
plt.plot([left, right], [0, 0], color='yellow', lw=10, zorder=1)
plt.scatter(pop_median, 0, color='green', s=60, zorder=2)

Does the interval contain the population median? We can tell in this case because we know the population value, but in general, we won't know what it is, so we'd just need to have some level of confidence that the interval contains the population median. With the interval we've created here, we'd be 95% confident that our interval contains the population value. (What a confidence interval really means is that 95% of the confidence intervals constructed from the same population, same sample size, in a similar way, will contain the true population value. There **isn't** necessarily a 95% probability that our specific confidence interval contains the mean, hence why we use the term "confidence." We will demonstrate this below.

## Theoretical Scenario: Creating Many Confidence Intervals

What does it mean to be "95% confident"? One way of thinking about it is that it's our level of confidence that the method we used to generate the confidence interval will actually contain the population value. Remember, our goal is to create an interval in which we're fairly certain that the population value lies. However, because of randomness in our data, we might end up with a sample that's unusual and gives a bad estimate of the population. 

In this section, we'll explore what might have happened if we were to draw 100 different samples and generate bootstrap confidence intervals using those samples. 

<font color = 'red'>**Question 5: Define a function called `bootstrap_confidence_interval` that takes in one original sample (`original_sample`), the number of bootstrap samples that we want to take (`num_repetitions`), and the confidence level as a percentage (`confidence_level`). This function should return an array of two values: the left bound and the right bound. Test this function out by using it with our sample using `num_repetitions = 500` and `confidence_level = 95`.**</font>

*Hint: Refer to your answers to questions 3 and 4. You can find the percentiles for the left and right bounds using the below formula:*
- left bound percentile: `(100-confidence_level)/2` 
- right bound percentile: `100 - (100 - confidence_level)/2`

In [ ]:
def bootstrap_confidence_interval(original_sample, num_repetitions, confidence_level):
    ...

bootstrap_confidence_interval(our_sample, 500, 95)

The `bootstrap_confidence_interval` function is what we can use to calculate our confidence interval with our original (non-bootstrap) sample. What would have happened if we had, for example, gone out into the world and collected 100 of these samples? Would the 90% confidence interval always contain the true population value? Let's find out. 

<font color = 'red'>**Question 6: Create a Table called `intervals` that contains the left and right bounds of each confidence interval that you might have created if you had gone out and collected a sample 100 times. Use a sample size of 300 for each "original sample". Use 500 bootstrap samples for each iteration.**</font>

*Hint: The `original_sample` should be a sample without replacement from the full population. Remember, we are trying to see what would have happened if we went out an interviewed 300 people 100 times. Use `population_salary` (rather than `population`) to make this code run faster.*

*Note: Much of the set up is provided below. Remember that we are using a 90% confidence interval here, not a 95% confidence interval.*

*This might take a little bit of time! Don't worry, that's normal. We are doing 100 iterations of taking 500 bootstrap samples.*


In [ ]:
left_ends = ds.make_array()
right_ends = ds.make_array()

for i in np.arange(100):
    original_sample = ...
    confidence_interval = ...
    left_ends = np.append(left_ends, ...)
    right_ends = np.append(right_ends, ...)

intervals = ds.Table().with_columns(
    'Left', left_ends,
    'Right', right_ends
) 

intervals.show(3)

Using this `interval` Table, we can now see how many times our confidence intervals contained the population median.

<font color = 'red'>**Question 7: What proportion of the time did the confidence interval contain the population value?**</font>

*Hint*: You can use `.where()` with `intervals` twice to find the cases in which interval contained the population median. 

<font color = 'red'>**Question 8: What do you think would have happened if we used an 80% confidence interval?**</font>

This shows a representation of what it means to be "90% confident". If 100 different people performed the same process of surveying 300 people, finding the sample median, and constructing a 90% confidence interval using their sample, we would expect about 90 of those confidence interval to contain the population median. 

In this way, there is still a small chance of making an error. We can construct a confidence interval that we are 90% (or 95% or 80% or 99%) confident contains the true population value, but we can't be absolutely sure. However, this gives us a way of quantifying that error.

## Using Total Compensation

Now, let's try creating a confidence interval for a different variable. We can take a look at the `Total Compensation` variable. 

In [ ]:
pop_median = np.median(population.column('Total Compensation'))
sf_bins = np.arange(0, 700000, 25000)
population.hist('Total Compensation', bins=sf_bins)
print("Population Median = ", pop_median)

In [ ]:
compensation_sample = population.sample(300, with_replacement = False).select('Total Compensation')
compensation_sample.show(5)

<font color = 'red'>**Question 9: Using `compensation_sample`, construct a 99% confidence interval for the `Total Compensation`. Interpret the confidence interval in context.**</font>

*Note: Remember, you shouldn't do all of the steps we did above in the "Theoretical Scenario" section. That was just to see what might happen if we were to create many confidence intervals to explore how we should think about confidence intervals. Normally, you just create one confidence interval.*

*Note 2: Use the median function above but now change the variable from "Total Salary" to "Total Compensation".  This is why we create functions!*